In [1]:
import numpy as np
from ultralytics import YOLO
import torch
import pandas as pd

print(torch.cuda.is_available())

True


In [2]:
# Predict with the model
# Load a model
model = YOLO('yolov8x-pose.pt')  # load an official model

source_vid = 'Videos/100_Thannheimer Germana_lq.mp4'
# source_vid = 'Videos/BUCHMANN2 Anne.mp4'

tracking_results = model.track(source=source_vid, conf=0.5, save=True, show=True)  # tracking on video


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs
video 1/1 (frame 1/111) C:\Users\Florian\PycharmProjects\CCA\Videos\100_Thannheimer Germana_lq.mp4: 384x640 2 persons, 127.7ms
video 1/1 (frame 2/111) C:\Users\Florian\PycharmProjects\CCA\Videos\100_Thannheimer Germana_lq.mp4: 384x640 2 persons, 22.5ms
video 1/1 (frame 3/111) C:\Users\Florian\PycharmProjects\CCA\Videos\100_Thannheimer Germana_lq.mp4: 384x640 1 person, 22.0ms
video 1/1 (frame 4/111) C:\Users\Florian\PycharmProjects\CCA\Videos\100_Thannheime

In [33]:
frame_numbers = []
ids = []
bboxes = []
kps = []
kp_confs = []

In [34]:
# Assume `video_results` is a list where each element represents the results for a frame
for frame_index, results in enumerate(tracking_results):
    for r in results:
        # Extract data for the current frame
        
        
        box_id = r.boxes.id.numpy() if r.boxes.id is not None else [0]
        bbox = r.boxes.xywh.cpu().numpy()
        kp = r.keypoints.xy.cpu().numpy()
        kp_conf = r.keypoints.conf.cpu().numpy()
        
        # Append data to lists
        frame_numbers.extend([frame_index] * len(box_id))
        ids.extend(box_id)
        bboxes.extend(bbox)
        kps.extend(kp)
        kp_confs.extend(kp_conf)
        

204 204 204 204 204


In [35]:
# Create DataFrame
df = pd.DataFrame({
    'FrameNumber': frame_numbers,
    'ID': ids,
    'BoundingBox': list(map(tuple, bboxes)),  # Convert arrays to tuples for better handling
    'Keypoints': list(map(tuple, kps)),
    'Keypoint Conf': list(map(tuple, kp_confs))
})

In [36]:
# Set multi-index for easier data retrieval by frame and ID
df.set_index(['FrameNumber', 'ID'], inplace=True)

In [37]:
# Now you can easily access all data related to a specific frame and ID
print(df.head())

                                                 BoundingBox  \
FrameNumber ID                                                 
0           0.0                 (1631.0, 547.0, 88.0, 170.0)   
            0.0                 (1705.5, 544.0, 69.0, 176.0)   
1           0.0                 (1630.0, 546.5, 86.0, 169.0)   
            0.0                 (1704.5, 544.5, 69.0, 177.0)   
2           1.0   (1628.6687, 546.0887, 83.59192, 168.18259)   
...                                                      ...   
47          2.0  (857.43097, 603.631, 122.317505, 231.66156)   
            3.0    (1648.7988, 544.714, 70.16589, 170.69962)   
48          2.0  (878.55664, 599.6742, 120.28955, 240.23874)   
            3.0    (1646.458, 544.5393, 68.42749, 169.55365)   
49          2.0  (898.9127, 595.92834, 115.56525, 249.28912)   

                                                         Keypoints  \
FrameNumber ID                                                       
0           0.0  ([0.0, 0.0

In [38]:
def calculate_distance_travelled(box1, box2):
    """ Calculate Euclidean distance between the centroids of two bounding boxes. """
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    centroid1 = (x1 + w1 / 2, y1 + h1 / 2)
    centroid2 = (x2 + w2 / 2, y2 + h2 / 2)
    return np.sqrt((centroid2[0] - centroid1[0]) ** 2 + (centroid2[1] - centroid1[1]) ** 2)


In [40]:
# Initialize a dictionary to store distances for each ID
distances = {}

# Get unique IDs
unique_ids = df.index.get_level_values('ID').unique()

# Loop over each unique ID
for uid in unique_ids:
    
    if uid == 0:
        distances[uid] = 0
    else:
        id_data = df.xs(uid, level='ID')
        first_frame = id_data.index.min()
        last_frame = id_data.index.max()
    
        # Get bounding box data for the first and last appearances
        first_bbox = id_data.loc[first_frame, 'BoundingBox']
        last_bbox = id_data.loc[last_frame, 'BoundingBox']
    
        # Calculate the distance traveled by the centroid
        distance = calculate_distance_travelled(first_bbox, last_bbox)
        distances[uid] = distance

# Find the ID with the maximum distance traveled
max_distance_id = max(distances, key=distances.get)

print(f"ID {max_distance_id} has traveled the longest distance of {distances[max_distance_id]} units.")

ID 2.0 has traveled the longest distance of 1870.5769362682431 units.


In [41]:
# Number of keypoints (COCO model has 17 keypoints)
num_kps = 17

# Create column names for the DataFrame
columns = ['Posx', 'Posy', 'width', 'height'] + \
          [f'PosX Kp{i+1}' for i in range(num_kps)] + \
          [f'PosY Kp{i+1}' for i in range(num_kps)] + \
          [f'Conf Kp{i+1}' for i in range(num_kps)] + \
          ['Label']

In [42]:
# Initialize DataFrame for ID 4 with 140 frames (assuming 140 is the number of frames)
xcs_df = pd.DataFrame(index=range(len(tracking_results)), columns=columns)

# Get data for ID 4 from the original DataFrame
xcs_data = df.xs(max_distance_id, level='ID')

# Loop through all frames where ID 4 is detected and fill the DataFrame
for frame in xcs_data.index:
    bbox = xcs_data.loc[frame, 'BoundingBox']
    kps = xcs_data.loc[frame, 'Keypoints']
    kp_confs = xcs_data.loc[frame, 'Keypoint Conf']
    
    # Fill bounding box data
    xcs_df.loc[frame, ['Posx', 'Posy', 'width', 'height']] = bbox
    
    # Fill keypoint data
    for i in range(num_kps):
        x_pos, y_pos = kps[i]
        
        xcs_df.loc[frame, [f'PosX Kp{i+1}']] = x_pos
        xcs_df.loc[frame, [f'PosY Kp{i+1}']] = y_pos
        xcs_df.loc[frame, [f'Conf Kp{i+1}']] = kp_confs[i]
        
# Fill missing data for frames where ID 4 isn't detected
xcs_df.fillna(0, inplace=True)  # Replace None with 0 or use np.nan to keep as NaN
xcs_df = xcs_df.infer_objects()


C:\Users\Florian\AppData\Local\Temp\ipykernel_23888\1610744318.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  xcs_df.fillna(0, inplace=True)  # Replace None with 0 or use np.nan to keep as NaN


In [43]:
import os

def get_filepaths(directory, extension=None):
    """
    Retrieves the file paths of all files in the specified directory,
    optionally filtering by file extension. If only one file is found, 
    returns a single string. Otherwise, returns a list of file paths.

    :param directory: The directory from which to list file paths.
    :param extension: Optional. Specify the file extension to filter by (e.g., 'json').
                      The extension should be provided without a dot.
    :return: A single file path as a string if only one file is found, 
             or a list of file paths if multiple files are found.
    """

    # List to store file paths
    file_paths = []

    # List all files and directories in the specified path
    for filename in os.listdir(directory):
        # Construct full file path
        file_path = os.path.join(directory, filename)
        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            # If an extension is specified, filter files by the extension
            if extension and filename.endswith('.' + extension):
                file_paths.append(file_path)
            elif not extension:  # If no extension is specified, add all files
                file_paths.append(file_path)

    # Return a single file path as a string if only one is found, otherwise return the list
    if len(file_paths) == 1:
        return file_paths[0]
    else:
        return file_paths

In [46]:
import json

# json_file = get_filepaths('Frames/BUCHMANN2 Anne/', extension='json')
json_file = get_filepaths('Frames/100_Thannheimer Germana_lq/', extension='json')

print(json_file)

with open(json_file, 'r') as file:
    label_info_full = json.load(file)

# Extract only the first five items
label_info = dict(list(label_info_full.items())[:5])

# Process the JSON data and update DataFrame
for label, path in label_info.items():
    # Extract the frame number from the file path
    frame_number = int(path.split('\\')[-1].replace('.png', ''))

    print(frame_number)

    # Update the label in the DataFrame if the frame number exists in the index
    if frame_number in xcs_df.index:
        xcs_df.loc[frame_number, ['Label']] = int(label)


Frames/100_Thannheimer Germana_lq/labels.json
23
25
31
40
47


In [49]:
# Display the DataFrame
print(xcs_df.head(112))

          Posx        Posy      width      height   PosX Kp1  PosX Kp2  \
0     0.000000    0.000000   0.000000    0.000000   0.000000       0.0   
1     0.000000    0.000000   0.000000    0.000000   0.000000       0.0   
2     0.000000    0.000000   0.000000    0.000000   0.000000       0.0   
3    24.730474  604.196899  49.396984  203.183807  41.533485       0.0   
4    32.508453  601.737305  64.896744  203.790436   0.000000       0.0   
..         ...         ...        ...         ...        ...       ...   
106   0.000000    0.000000   0.000000    0.000000   0.000000       0.0   
107   0.000000    0.000000   0.000000    0.000000   0.000000       0.0   
108   0.000000    0.000000   0.000000    0.000000   0.000000       0.0   
109   0.000000    0.000000   0.000000    0.000000   0.000000       0.0   
110   0.000000    0.000000   0.000000    0.000000   0.000000       0.0   

      PosX Kp3  PosX Kp4   PosX Kp5  PosX Kp6  ...  Conf Kp9  Conf Kp10  \
0     0.000000       0.0   0.000000 

In [50]:
xcs_df.to_csv('feature_extraction.csv', index=False)

In [44]:
from functions import get_matching_files, get_filepaths, rm_file_extension

frames_dir = "Frames"
videos_dir = "Videos"

get_filepaths()

path_without = rm_file_extension(source_vid)

print(path_without)
matches, frames_subdirectories = get_matching_files(frames_dir, videos_dir)
# 
print(frames_subdirectories)
# 
# # for subdir in frames_subdirectories:
#     
#     joint_directory = os.path.join(frames_dir, frames_subdirectories)
# 
# print(joint_directory)
# 
# json_file = get_filepaths(joint_directory, extension='json')


Videos/BUCHMANN2 Anne
['100_Thannheimer Germana_lq', '101_Veit Verena_lq', '102_Ott Louisa_lq', '103_Schmidt Henriette_lq', '104_Dietze Alexandra_lq', '105_Saunus Annika_lq', '106_Haberstumpf Katharina_lq', '107_Zeissler Jule_lq', '108_Dietze Anna_lq', '109_Rekowski Bente_lq', '10_Moosmayer Moritz_lq', '110_Niederacher Agnes_lq', '111_Stocker Hannah_lq', '112_Uschold Fredericka_lq', '113_Wehrle Amelie_lq', '114_Scholz Sina_lq', '116_Gerg Theresa_lq', '118_Eisenlauer Sebastian_lq', '119_Maettig Valentin_lq', '11_Schlegelmilch Felix_lq', '120_Leupold Richard_lq', '121_Leismueller Jakob_lq', '122_Foettinger Michael_lq', '123_Spoetzl Thomas_lq', '124_Schubert Felian_lq', '125_Struebel Josua_lq', '126_Faessler Josef_lq', '127_Bergelt Franz_lq', '128_Heinrich Tom_lq', '129_Wetterling Felix_lq', '12_Goedecke Luis_lq', '131_Hartig Fabian_lq', '133_Knopf Florian_lq', '135_Schroeter Jonas_lq', '136_Moch Friedrich_lq', '137_Bauroth Moritz_lq', '138_Werner Jonah_lq', '139_Bauer Marius_lq', '140_So

In [4]:
import os

frames_dir = 'Frames'

def simplify_filename(filepath):
    # Extract the filename without any directory path
    filename = os.path.basename(filepath)
    # Remove the file extension
    filename_without_extension, _ = os.path.splitext(filename)
    return filename_without_extension

# Example usage:
file_path = 'Videos/BUCHMANN2 Anne.mp4'
simplified_name = simplify_filename(file_path)
print(simplified_name)

json_file = os.path.join(frames_dir, simplified_name, 'labels.json')

print(json_file)

BUCHMANN2 Anne
Frames\BUCHMANN2 Anne\labels.json
